
# Advent of Code 2023

> The effort of using machines to mimic the human mind has always struck me as rather silly. I would rather use them to mimic something better.

-- Edsger W. Dijkstra

## Imports and definitions

In [138]:
#type: ignore
from functools import reduce
from itertools import product
from operator import mul, or_
from math import gcd
from dataclasses import dataclass
from collections import Counter, defaultdict
import re


def prod(u): 
    """Older Pythons don't have this."""
    return reduce(mul, u)


def lcm(*u):
    """Older Pythons don't have this."""
    return reduce(lambda a, b: a * b // gcd(a, b), u)


def inputfunc(day, kind='lines', testing=False):
    """Generator to read input files."""
    filename = 'test.txt' if testing else f"input/{day}.txt"

    def gen(func):
        if kind == 'lines':
            text = [x.strip() for x in open(filename)]
        elif kind == 'chunks':
            text = [
                x.strip()
                for x in open(filename).read().split('\n\n')
                if x.strip()
            ]
        elif kind == 'single':
            text = open(filename).read().strip()
        elif kind == 'commas':
            text = [x.strip() for x in open(filename).read().split(',')]
        elif kind == 'raw':
            text = open(filename).read()

        def inner():
            return func(f=text)
        return inner
    return gen

## [Day 1 - Trebuchet?!](https://adventofcode.com/2023/day/1)

In [3]:
@inputfunc(1)
def input_1(f):
    return f


def find_digits(s):
    return [int(i) for i in s if i.isdigit()]


A = sum(
    10 * x[0] + x[-1]
    for x in (find_digits(s) for s in input_1())
)

assert A == 55123


d = {
    'one': 1,
    'two': 2,
    'three': 3,
    'four': 4,
    'five': 5,
    'six': 6,
    'seven': 7,
    'eight': 8,
    'nine': 9
}

r = re.compile(f'(?=(\\d|{"|".join(d)}))')

def find_digit_names(s):
    def to_number(p):
        if p.isdigit():
            return int(p)
        else:
            return d[p]

    return [to_number(p) for p in r.findall(s)]


A = sum(
    10 * x[0] + x[-1]
    for x in (find_digit_names(s) for s in input_1())
)

assert A == 55260

## [Day 2 - Cube Conundrum](https://adventofcode.com/2023/day/2)

In [83]:
@dataclass
class Game:
    gameid: int
    contents: list[Counter[str, int]]

        
@inputfunc(2)
def input_2(f):
    for l in f:
        num, items = l.split(':')
        
        _, gameid = num.split()
        
        contents = [
            Counter({
                b: int(a)
                for a, b in [
                    y.split()
                    for y in x.split(',')
                ]
            })
            for x in items.split(';')
        ]
        
        yield Game(int(gameid), contents)


games = list(input_2())


cubes = Counter({'red': 12, 'green': 13, 'blue': 14})
A = sum(
    game.gameid 
    for game in games
    if all(
        d < cubes for d in game.contents
    )
)

assert A == 2476


A = sum(
    prod(
        reduce(or_, game.contents).values()
    )
    for game in games
)

assert A == 54911

## [Day 3 - Gear Ratios](https://adventofcode.com/2023/day/3)

In [142]:
@inputfunc(3)
def input_3(f):
    return [
        list(x) for x in f
    ]
    
    
class Grid:
    def __init__(self, g):
        self._grid = g
        self.columns = len(g[0])
        self.rows = len(g)

    def __getitem__(self, t):
        x, y = t
        return self._grid[y][x]
    
    def adjacent(self, x, y):
        return [
            self[i, j] 
            for i, j in {
                (x+1, y), (x-1, y), (x, y+1), (x, y-1),
                (x+1, y+1), (x-1, y+1), (x+1, y-1), (x-1, y-1)
            }
            if 0 <= i < self.columns and 0 <= j < self.rows
        ]
    

@dataclass
class PartNumber:
    value: int
    col_start: int
    col_end: int
    row: int


grid = Grid(input_3())


def find_part_numbers(grid, adjacent_to=None):
    val, symbol_adjacent = 0, False
    
    def is_symbol(s):
        if adjacent_to:
            return s in adjacent_to
        else:
            return not (s.isdigit() or s == '.')

    for j in range(grid.rows):
        for i in range(grid.columns):
            c = grid[i, j]
            if c.isdigit():
                val = 10 * val + int(grid[i, j])
            if not c.isdigit():
                if val and symbol_adjacent:
                    yield PartNumber(
                        val, i - len(str(val)), i - 1, j
                    )
                val, symbol_adjacent = 0, False
            if val and any(is_symbol(s) for s in grid.adjacent(i, j)):
                symbol_adjacent = True
        if val and symbol_adjacent:
            yield PartNumber(
                val, grid.columns - len(str(val)), grid.columns - 1, j
            )
        val, symbol_adjacent = 0, False
            
            
A = sum(p.value for p in find_part_numbers(grid))
assert A == 536576


def find_gear_ratios(grid):
    part_numbers = defaultdict(list)
    for p in find_part_numbers(grid, '*'):
        part_numbers[p.row].append(p)
    
    for i, j in product(range(grid.columns), range(grid.rows)):
        if grid[i, j] != '*':
            continue
                
        adjacents = (
            [
                p for p in part_numbers[j] 
                if p.col_start == i+1 or p.col_end == i-1
            ] +
            [
                p for p in part_numbers[j-1] + part_numbers[j+1]
                if p.col_start <= i+1 and p.col_end >= i-1
            ]
        )
            
        if len(adjacents) == 2:
            yield adjacents
            
    
A = sum(a.value * b.value for a, b in find_gear_ratios(grid))
assert A == 75741499